In [1]:
import pandas as pd
import numpy as np
from google.colab import files
import string
import io
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from functools import reduce
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import PorterStemmer
import math
import re
import seaborn as sns
from copy import copy, deepcopy
from unicodedata import normalize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from numpy import asarray

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
file = open('drive/My Drive/spa.txt').read().split('\n')

In [3]:
def My_train_test_split(df,df2,train_percent, test_percent):
    # calculate random permutation of indices
    m = len(df)
    perm = np.random.permutation(m)
    train_end = int(train_percent * m)
    test_end = int(test_percent * m) + train_end
    
    Xtrain = [df[i] for i in perm[:train_end]]
    Xtest = [df[i] for i in perm[train_end:test_end]]
    
    Ytrain = [df2[i] for i in perm[:train_end]]
    Ytest = [df2[i] for i in perm[train_end:test_end]]
    return(Xtrain, Xtest, Ytrain,Ytest,)  
  
def clean_pairs(lines):
  cleaned = list()
  # prepare regex for char filtering
  re_print = re.compile('[^%s]' % re.escape(string.printable))
  # prepare translation table for removing punctuation
  table = str.maketrans('', '', string.punctuation)
  for pair in lines:
    clean_pair = list()
    for line in pair:
      # rmove word characters.
      line = normalize('NFD', line).encode('ascii', 'ignore')
      line = line.decode('UTF-8')
      # tokenize on white space
      line = line.split()
      # convert to lowercase
      line = [word.lower() for word in line]
      # remove punctuation from each token
      line = [word.translate(table) for word in line]
      # remove non-printable chars form each token
      line = [w.strip() for w in line]
      # remove tokens with numbers in them
      line = [word for word in line if word.isalpha()]
      # store as string
      clean_pair.append(' '.join(line))
    cleaned.append(clean_pair)
  return(cleaned)

lines = file[0:7500]
split = [re.split(r'\t+', item)for item in lines]
split = clean_pairs(split)
X  = [x[0] for x in split]
Y  = ['START_ ' + x[1] + ' _END'for x in split]
#Y = [y.split() for y in Y]

df = pd.DataFrame(
    {'X': X,
     'Y': Y,
    })

df = df.drop_duplicates(subset=['X'], keep= 'first')

print(df)
X = df.X.values
X  = [x.split() for x in X]
Y = df.Y.values
Y  = [y.split() for y in Y]


                    X                                      Y
0                  go                         START_ ve _END
4                  hi                       START_ hola _END
5                 run                      START_ corre _END
7                 who                      START_ quien _END
8                 wow                      START_ orale _END
9                fire                      START_ fuego _END
12               help                      START_ ayuda _END
15               jump                      START_ salta _END
17               stop                      START_ parad _END
20               wait                     START_ espera _END
22              go on                   START_ continua _END
24              hello                       START_ hola _END
25              i ran                      START_ corri _END
27              i try                 START_ lo intento _END
28              i won                  START_ he ganado _END
29              oh no   

In [0]:
all_eng_words  = set([item for sublist in X for item in sublist])
all_spa_words = set([item for sublist in Y for item in sublist])

eng_sentence_length = max([len(sample.split()) for sample in df.X])
spa_sentence_length = max([len(sample.split()) for sample in df.Y])

input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_spa_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_spa_words)

input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

encoder_input_data = np.zeros(
    (len(df.X), eng_sentence_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(df.Y), spa_sentence_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(df.Y), spa_sentence_length, num_decoder_tokens),
    dtype='float32')

In [0]:
for i, (input_text, target_text) in enumerate(zip(df.X, df.Y)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            
            


In [0]:
embeddings_index = dict()
f = open('drive/My Drive/glove.6B.100d.txt').readlines()
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs

MAX_NUM_WORDS = 1880
EMBEDDING_DIM = embeddings_index.get('a').shape[0]
num_words = min(MAX_NUM_WORDS, len(all_eng_words)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for i, word in enumerate(list(all_eng_words)):
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word) ## This references the loaded embeddings dictionary
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [12]:
embedding_layer = Embedding(num_encoder_tokens,50,weights=[embedding_matrix],input_length=eng_sentence_length)
print(len(embedding_matrix))

1881


In [13]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model
from keras.utils import plot_model

embedding_size = 100
encoder_inputs = Input(shape=(None,))
#en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
en_x = Embedding(num_encoder_tokens, embedding_size, weights= [embedding_matrix], trainable=False)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dex=  Embedding(num_decoder_tokens, embedding_size)
final_dex= dex(decoder_inputs)
decoder_lstm = LSTM(50, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 100)    188100      input_2[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 100)    293600      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [14]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=128,
          epochs=50,
          validation_split=0.05)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 5042 samples, validate on 266 samples
Epoch 1/50
5042/5042 [==============================] - 12s 2ms/step - loss: 2.5206 - acc: 0.0665 - val_loss: 2.3011 - val_acc: 0.1000
Epoch 2/50
5042/5042 [==============================] - 10s 2ms/step - loss: 2.0129 - acc: 0.1000 - val_loss: 2.1610 - val_acc: 0.1000
Epoch 3/50
5042/5042 [==============================] - 10s 2ms/step - loss: 1.9115 - acc: 0.1000 - val_loss: 2.1353 - val_acc: 0.1000
Epoch 4/50
5042/5042 [==============================] - 10s 2ms/step - loss: 1.8598 - acc: 0.1000 - val_loss: 2.1158 - val_acc: 0.1000
Epoch 5/50
5042/5042 [==============================] - 10s 2ms/step - loss: 1.8050 - acc: 0.1018 - val_loss: 2.0821 - val_acc: 0.1132
Epoch 6/50
5042/5042 [==============================] - 10s 2ms/step - loss: 1.7533 - acc: 0.1094 - val_loss: 2.0509 - val_acc: 0.1132
Epoch 7/50
5042/5

In [18]:
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, None, 100)         188100    
_________________________________________________________________
lstm_1 (LSTM)                [(None, 50), (None, 50),  30200     
Total params: 218,300
Trainable params: 30,200
Non-trainable params: 188,100
_________________________________________________________________


In [0]:
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

final_dex2= dex(decoder_inputs)

decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [19]:
from nltk.translate.bleu_score import sentence_bleu

for seq_index in [30,22,35,500, 700, 800,5000,4500,3000,2000]:
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', df.X[seq_index: seq_index + 1])
    print('Decoded sentence:', decoded_sentence)


-
Input sentence: 47    i quit
Name: X, dtype: object
Decoded sentence:  tom se se quedo _END
-
Input sentence: 39    he ran
Name: X, dtype: object
Decoded sentence:  el se quedo _END
-
Input sentence: 53    listen
Name: X, dtype: object
Decoded sentence:  son caja _END
-
Input sentence: 850    wait here
Name: X, dtype: object
Decoded sentence:  aqui aqui _END
-
Input sentence: 1157    im better
Name: X, dtype: object
Decoded sentence:  estoy cansado _END
-
Input sentence: 1314    quiet down
Name: X, dtype: object
Decoded sentence:  vuelve a casa _END
-
Input sentence: 7116    tom is useless
Name: X, dtype: object
Decoded sentence:  tom es tom _END
-
Input sentence: 6440    it looks great
Name: X, dtype: object
Decoded sentence:  se es bien _END
-
Input sentence: 4453    i said enough
Name: X, dtype: object
Decoded sentence:  yo dije que dije _END
-
Input sentence: 3040    ive lost it
Name: X, dtype: object
Decoded sentence:  lo lo he hecho _END


In [0]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np

def softmax(x):
  e = K.exp(x - K.max(x, axis=1, keepdims=True))
  s = K.sum(e, axis=1, keepdims=True)
  return e / s


repeator = RepeatVector(eng_sentence_length)
concatenator = Concatenate(axis=-1)
densor = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)

In [0]:
def one_step_attention(a, s_prev):    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    concat = concatenator([a, s_prev])
    # Use densor to propagate concat through a small fully-connected neural network to compute the "energies" variable e. (≈1 lines)
    e = densor(concat)
    # Use activator and e to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(e)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas, a])
    ### END CODE HERE ###  
    return context
  
  
n_a = 64
n_s = 128
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(num_decoder_tokens, activation=softmax)

In [0]:
def model(Tx, Ty, n_a,n_s, human_vocab_size, machine_vocab_size):
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 and c0, initial hidden state for the decoder LSTM of shape (n_s,)
    X = Input(shape=(Tx,human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. Remember to use return_sequences=True. (≈ 1 line)
    a = Bidirectional(LSTM(n_a, return_sequences=True))(X)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model([X, s0, c0], outputs)
    
    ### END CODE HERE ###
    
    return model

In [0]:
model = model(eng_sentence_length, spa_sentence_length,n_a,n_s, num_encoder_tokens, num_decoder_tokens)


In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 5, 1881)      0                                            
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 5, 128)       996352      input_7[0][0]                    
__________________________________________________________________________________________________
repeat_vector_2 (RepeatVector)  (None, 5, 128)       0           s0[0][0]                         
                                                                 lstm_7[0][0]                     
          

In [0]:
#opt = Adam(lr = 0.005, beta_1=0.9, beta_2=0.999, decay = 0.01)
#model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:

input_words  = set([item for sublist in X for item in sublist])
target_words = set([item for sublist in Y for item in sublist])
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)


max_encoder_seq_length = max([len(sample) for sample in X])
max_decoder_seq_length = max([len(sample) for sample in Y])


input_token_index = dict(
  [(char, i) for i, char in enumerate(input_words)])
target_token_index = dict(
  [(char, i) for i, char in enumerate(target_words)])

encoder_input_data = np.zeros(
  (len(X), max_encoder_seq_length, num_encoder_tokens),
  dtype='float32')
decoder_input_data = np.zeros(
  (len(X), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')
decoder_target_data = np.zeros(
  (len(X), max_decoder_seq_length, num_decoder_tokens),
  dtype='float32')

for i, (X, Y) in enumerate(zip(X, Y)):
  for t, char in enumerate(X):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  for t, char in enumerate(Y):
    # decoder_target_data is ahead of decoder_input_data by one timestep
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      # decoder_target_data will be ahead by one timestep
      # and will not include the start character.
      decoder_target_data[i, t - 1, target_token_index[char]] = 1.

s0 = np.zeros((5308, n_s))
c0 = np.zeros((5308, n_s))

outputs = list(decoder_input_data.swapaxes(0,1))


In [0]:
print(encoder_input_data.shape)
model.fit([encoder_input_data,s0,c0], outputs, epochs=20, batch_size=128)

(5308, 5, 1881)
Epoch 1/20
5308/5308 [==============================] - 36s 7ms/step - loss: 29.4845 - dense_6_loss: 1.7511e-04 - dense_6_acc: 0.1319 - dense_6_acc_1: 0.0000e+00 - dense_6_acc_2: 0.0599 - dense_6_acc_3: 0.3223 - dense_6_acc_4: 0.4050 - dense_6_acc_5: 0.1558 - dense_6_acc_6: 0.0260 - dense_6_acc_7: 0.0058 - dense_6_acc_8: 7.5358e-04 - dense_6_acc_9: 1.8839e-04
Epoch 2/20
5308/5308 [==============================] - 20s 4ms/step - loss: 20.6956 - dense_6_loss: 2.2180e-04 - dense_6_acc: 1.0000 - dense_6_acc_1: 0.0000e+00 - dense_6_acc_2: 0.0607 - dense_6_acc_3: 0.3318 - dense_6_acc_4: 0.4154 - dense_6_acc_5: 0.1590 - dense_6_acc_6: 0.0264 - dense_6_acc_7: 0.0058 - dense_6_acc_8: 7.5358e-04 - dense_6_acc_9: 1.8839e-04
Epoch 3/20
5308/5308 [==============================] - 20s 4ms/step - loss: 19.1619 - dense_6_loss: 1.3321e-04 - dense_6_acc: 1.0000 - dense_6_acc_1: 0.0121 - dense_6_acc_2: 0.0607 - dense_6_acc_3: 0.3318 - dense_6_acc_4: 0.4154 - dense_6_acc_5: 0.1590 - dens

KeyboardInterrupt: ignored

In [0]:
for ele in [30,22,35,500, 700, 800,5000,4500,3000,2000]:
  input_seq = encoder_input_data[ele: ele + 1]
  prediction = model.predict([input_seq, s0, c0])
  prediction = np.argmax(prediction, axis = -1)
  output = [reverse_target_char_index[int(i)] for i in prediction]
  print(output)


['blanca', 'mentiroso', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton']
['blanca', 'dedico', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton']
['blanca', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton']
['blanca', 'mentiroso', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton']
['blanca', 'preguntare', 'murcielago', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton']
['blanca', 'caminando', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton']
['blanca', 'impaciente', 'dedico', 'impaciente', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton']
['blanca', 'dedico', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton']
['blanca', 'preguntare', 'murcielago', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton', 'monton']
['blanca', 'caminando', 'monton', 'monton', 'monton', 'mon

In [0]:
from keras.layers import SimpleRNN

encoder_input = Input(shape=(eng_sentence_length,))
encoder = Embedding(num_encoder_tokens, 64, input_length=eng_sentence_length, mask_zero=True)(encoder_input)
encoder = LSTM(64, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]

print('encoder', encoder)
print('encoder_last', encoder_last)

decoder_input = Input(shape=(spa_sentence_length,))
decoder = Embedding(num_decoder_tokens, 64, input_length=spa_sentence_length, mask_zero=True)(decoder_input)
decoder = LSTM(64, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])

print('decoder', decoder)

encoder Tensor("lstm_3/transpose_2:0", shape=(?, 5, 64), dtype=float32)
encoder_last Tensor("strided_slice:0", shape=(?, 64), dtype=float32)
decoder Tensor("lstm_4/transpose_2:0", shape=(?, 10, 64), dtype=float32)


In [0]:
from keras.layers import Activation, dot, concatenate
from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense

# Equation (7) with 'dot' score from Section 3.1 in the paper.
# Note that we reuse Softmax-activation layer instead of writing tensor calculation
attention = dot([decoder, encoder], axes=[2, 2])
attention = Activation('softmax', name='attention')(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

# Has another weight + tanh layer as described in equation (5) of the paper
output = TimeDistributed(Dense(64, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(num_decoder_tokens, activation="softmax"))(output)
print('output', output)

attention Tensor("attention/truediv:0", shape=(?, 10, 5), dtype=float32)
context Tensor("dot_2/MatMul:0", shape=(?, 10, 64), dtype=float32)
decoder_combined_context Tensor("concatenate_1/concat:0", shape=(?, 10, 128), dtype=float32)
output Tensor("time_distributed_2/Reshape_1:0", shape=(?, 10, 2936), dtype=float32)


In [0]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 5)            0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 10, 64)       187904      input_6[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 5, 64)        120384      input_5[0][0]                    
__________________________________________________________________________________________________
lstm_4 (LS

In [0]:
model.fit(x = [encoder_input_data, decoder_input_data],y = [decoder_target_data] ,verbose=2, batch_size=128, epochs=20)

Epoch 1/20
 - 8s - loss: 2.7981
Epoch 2/20
 - 9s - loss: 2.7504
Epoch 3/20
 - 9s - loss: 2.7058
Epoch 4/20
 - 8s - loss: 2.6659
Epoch 5/20
 - 9s - loss: 2.6232
Epoch 6/20
 - 9s - loss: 2.5825
Epoch 7/20
 - 9s - loss: 2.5422
Epoch 8/20
 - 9s - loss: 2.5006
Epoch 9/20
 - 8s - loss: 2.4586
Epoch 10/20
 - 9s - loss: 2.4211
Epoch 11/20
 - 9s - loss: 2.3842
Epoch 12/20
 - 8s - loss: 2.3486
Epoch 13/20
 - 9s - loss: 2.3130
Epoch 14/20
 - 9s - loss: 2.2814
Epoch 15/20
 - 8s - loss: 2.2484
Epoch 16/20
 - 9s - loss: 2.2167
Epoch 17/20
 - 9s - loss: 2.1894
Epoch 18/20
 - 8s - loss: 2.1598
Epoch 19/20
 - 8s - loss: 2.1311
Epoch 20/20
 - 8s - loss: 2.1044


In [0]:
def generate(text):
    encoder_input = text
    decoder_input = np.zeros(shape=(eng_sentence_length, spa_sentence_length))
    decoder_input[:,0] = target_token_index['START_']
    for i in range(1, spa_sentence_length):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return decoder_input[:,1:]

def decode(sequence):
  text = ''
  for i in sequence:
    if reverse_target_char_index[i] == '_END':
      break
    else:
      text += ' ' + reverse_target_char_index[i]
    print(reverse_target_char_index[i])
    
  return text

def to_katakana(text):
    decoder_output = generate(text)
    return decode(decoder_output[0])


to_katakana(encoder_input_data[500:501])
#decoder_input = np.zeros(shape=(eng_sentence_length, spa_sentence_length))
#decoder_input[:,0] = target_token_index['START_']
#model.predict([encoder_input_data[13:14], decoder_input]).argmax(axis=2)


tom
un
favor


' tom un favor'

In [0]:
https://github.com/wanasit/katakana/blob/master/notebooks/Attention-based%20Sequence-to-Sequence%20in%20Keras.ipynb
https://github.com/gowtham1997/machine_translation/blob/master/Neural%20machine%20translation%20with%20attention%20-%20v2.ipynb
https://github.com/naveenkumarg651/Neural_machine_translation-with-Attention/blob/master/attention_s2s_hindi.py